### Лабораторна робота №5, Обробка та аналіз текстових даних на Python, Варіант 14
**Виконав**: студент групи ІП-11, Лошак Віктор Іванович<br>
**Перевірив**: Юлія Тимофєєва Сергіївна<br>

**Тема роботи**: Моделювання тем<br>
**Мета роботи**: Ознайомитись з вирішенням задач пошуку ключових слів та моделювання тем.

17.03.2024

**Завдання**:<br>
1. Застосувати прихований розподіл Діріхле бібліотеки scikit-learn для моделювання тем. Обрати оптимальну кількість тем, оцінивши та порівнявши різні моделі. Вивести терми, що описують кожну з тем.
2. Використати текст chesterton-ball.txt з корпусу gutenberg бібліотеки nltk та вивести ключові біграми.

**Task**:<br>
1. Apply the hidden Dirichlet distribution of the scikit-learn library to model topics. Choose the optimal number of topics by evaluating and comparing different models. Derive terms describing each of the topics.
2. Use the text chesterton-ball.txt from the gutenberg corpus of the nltk library and output key bigrams.